# Claude Skills - Document Understanding com RAG

Este notebook demonstra como usar RAG (Retrieval Augmented Generation) para fazer perguntas sobre a documentação do Claude Skills do Anthropic.

## Setup e Importações

In [29]:
!pip install -q langchain langchain-community langchain-openai langchain-text-splitters langchain-core chromadb beautifulsoup4 python-dotenv tiktoken


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [30]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

## Carregando Documento do GitHub

Vamos carregar o README.md do repositório Claude Skills da Anthropic usando o WebBaseLoader.

In [31]:
from langchain_community.document_loaders import WebBaseLoader

# URL do README do Claude Skills
url = "https://raw.githubusercontent.com/anthropics/skills/main/README.md"

loader = WebBaseLoader(url)
docs = loader.load()

In [32]:
# Verificar o conteúdo carregado
print(f"Número de documentos carregados: {len(docs)}")
print(f"\nTamanho do documento: {len(docs[0].page_content)} caracteres")
print(f"\nPrimeiros 500 caracteres:\n{docs[0].page_content[:500]}")

Número de documentos carregados: 1

Tamanho do documento: 7632 caracteres

Primeiros 500 caracteres:
# Skills
Skills are folders of instructions, scripts, and resources that Claude loads dynamically to improve performance on specialized tasks. Skills teach Claude how to complete specific tasks in a repeatable way, whether that's creating documents with your company's brand guidelines, analyzing data using your organization's specific workflows, or automating personal tasks.

For more information, check out:
- [What are skills?](https://support.claude.com/en/articles/12512176-what-are-skills)
- 


## Divisão do Documento (Text Splitting)

Dividimos o documento em chunks menores para melhorar a recuperação de informações relevantes.
**MELHORIAS**: Chunks menores (800) com overlap maior (200) para manter mais contexto.

In [33]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Parâmetros otimizados para melhor recuperação
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,        # Reduzido de 1500 para 800
    chunk_overlap=200,     # Aumentado de 150 para 200
    separators=["\n\n", "\n", ". ", " ", ""],  # Prioriza quebras naturais
    length_function=len
)

splits = text_splitter.split_documents(docs)

In [34]:
print(f"Número de chunks criados: {len(splits)}")
print(f"\nPrimeiro chunk:\n{splits[0].page_content}")
print(f"\nÚltimo chunk:\n{splits[-1].page_content}")

Número de chunks criados: 12

Primeiro chunk:
# Skills
Skills are folders of instructions, scripts, and resources that Claude loads dynamically to improve performance on specialized tasks. Skills teach Claude how to complete specific tasks in a repeatable way, whether that's creating documents with your company's brand guidelines, analyzing data using your organization's specific workflows, or automating personal tasks.

Último chunk:
The markdown content below contains the instructions, examples, and guidelines that Claude will follow. For more details, see [How to create custom skills](https://support.claude.com/en/articles/12512198-creating-custom-skills).

# Partner Skills

Skills are a great way to teach Claude how to get better at using specific pieces of software. As we see awesome example skills from partners, we may highlight some of them here:

- **Notion** - [Notion Skills for Claude](https://www.notion.so/notiondevs/Notion-Skills-for-Claude-28da4445d27180c7af1df7d8615723d0)

## Embeddings

Vamos criar embeddings usando OpenAI para representar os chunks de texto em formato vetorial.

In [35]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-small")  # Modelo mais recente

## Criando o Vectorstore

Armazenamos os embeddings no ChromaDB para realizar buscas por similaridade.

In [36]:
from langchain_community.vectorstores import Chroma
import os

persist_directory = '../out/chroma_claude_skills/'

# Criar diretório se não existir
os.makedirs(persist_directory, exist_ok=True)

In [37]:
# Remove banco de dados antigo se existir e recria o diretório
!rm -rf ../out/chroma_claude_skills/
!mkdir -p ../out/chroma_claude_skills/
!chmod 777 ../out/chroma_claude_skills/

In [38]:
# Criar vectorstore com configurações específicas para evitar problemas de permissão
import chromadb
from chromadb.config import Settings

# Criar cliente com configurações personalizadas
chroma_client = chromadb.PersistentClient(
    path=persist_directory,
    settings=Settings(
        anonymized_telemetry=False,
        allow_reset=True
    )
)

# Criar collection
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory,
    client=chroma_client
)

InternalError: Query error: Database error: error returned from database: (code: 1032) attempt to write a readonly database

In [ ]:
print(f"Número de documentos no vectorstore: {vectordb._collection.count()}")

## Testes de Similarity Search

Agora vamos fazer perguntas sobre Claude Skills e validar se as respostas estão alinhadas com o contexto do documento original.
**MELHORIA**: Aumentado k=5 para recuperar mais documentos relevantes.

### Pergunta 1: O que é Claude Skills?

In [ ]:
question1 = "What is Claude Skills?"
docs_q1 = vectordb.similarity_search(question1, k=5)  # Aumentado de 3 para 5

print(f"Pergunta: {question1}")
print(f"\nNúmero de documentos recuperados: {len(docs_q1)}")
print("\n" + "="*80)

for i, doc in enumerate(docs_q1, 1):
    print(f"\nDocumento {i}:")
    print("-" * 80)
    print(doc.page_content)
    print("-" * 80)

### Pergunta 2: Como instalar Claude Skills?

In [ ]:
question2 = "How do I install Claude Skills?"
docs_q2 = vectordb.similarity_search(question2, k=5)

print(f"Pergunta: {question2}")
print(f"\nNúmero de documentos recuperados: {len(docs_q2)}")
print("\n" + "="*80)

for i, doc in enumerate(docs_q2, 1):
    print(f"\nDocumento {i}:")
    print("-" * 80)
    print(doc.page_content)
    print("-" * 80)

### Pergunta 3: Quais são as principais funcionalidades (skills) disponíveis?

In [ ]:
question3 = "What skills are available in Claude Skills?"
docs_q3 = vectordb.similarity_search(question3, k=5)

print(f"Pergunta: {question3}")
print(f"\nNúmero de documentos recuperados: {len(docs_q3)}")
print("\n" + "="*80)

for i, doc in enumerate(docs_q3, 1):
    print(f"\nDocumento {i}:")
    print("-" * 80)
    print(doc.page_content)
    print("-" * 80)

### Pergunta 4: Como contribuir para o projeto?

In [ ]:
question4 = "How can I contribute to Claude Skills?"
docs_q4 = vectordb.similarity_search(question4, k=5)

print(f"Pergunta: {question4}")
print(f"\nNúmero de documentos recuperados: {len(docs_q4)}")
print("\n" + "="*80)

for i, doc in enumerate(docs_q4, 1):
    print(f"\nDocumento {i}:")
    print("-" * 80)
    print(doc.page_content)
    print("-" * 80)

### Pergunta 5: Quais são os requisitos do sistema?

In [ ]:
question5 = "What are the system requirements for Claude Skills?"
docs_q5 = vectordb.similarity_search(question5, k=5)

print(f"Pergunta: {question5}")
print(f"\nNúmero de documentos recuperados: {len(docs_q5)}")
print("\n" + "="*80)

for i, doc in enumerate(docs_q5, 1):
    print(f"\nDocumento {i}:")
    print("-" * 80)
    print(doc.page_content)
    print("-" * 80)

## Validação com RAG Completo

Vamos usar um LLM para gerar respostas baseadas no contexto recuperado.
**MELHORIAS**: 
- Modelo GPT-4 para respostas mais precisas
- Prompt customizado para instruir o modelo
- Retriever com MMR para diversidade de documentos
- Usando LCEL (LangChain Expression Language) - abordagem moderna

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Usar GPT-4 para respostas mais precisas (ou gpt-3.5-turbo se preferir economizar)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Template de prompt customizado
template = """Use os seguintes trechos de contexto para responder a pergunta no final.
Se você não sabe a resposta com base no contexto fornecido, diga que não sabe, não tente inventar uma resposta.
Seja específico e detalhado na sua resposta, citando informações do contexto quando relevante.
Se a pergunta for sobre exemplos ou instruções, forneça-os de forma clara e estruturada.

Contexto:
{context}

Pergunta: {question}

Resposta detalhada:"""

prompt = ChatPromptTemplate.from_template(template)

# Usar MMR (Maximal Marginal Relevance) para diversidade nos documentos recuperados
retriever = vectordb.as_retriever(
    search_type="mmr",  # Mudança importante: MMR ao invés de similarity
    search_kwargs={
        "k": 5,          # Recuperar 5 documentos
        "fetch_k": 10    # Buscar 10 candidatos antes de aplicar MMR
    }
)

# Função auxiliar para formatar documentos
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Criar a chain usando LCEL (LangChain Expression Language)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### Testando com perguntas e validando respostas

In [ ]:
def ask_and_validate(question):
    """
    Faz uma pergunta ao sistema RAG e mostra a resposta com as fontes.
    """
    print("="*80)
    print(f"PERGUNTA: {question}")
    print("="*80)
    
    # Obter resposta
    answer = rag_chain.invoke(question)
    
    # Obter documentos fonte
    source_docs = retriever.invoke(question)
    
    print(f"\nRESPOSTA:\n{answer}")
    print(f"\n{'='*80}")
    print("DOCUMENTOS FONTE:")
    print("="*80)
    
    for i, doc in enumerate(source_docs, 1):
        print(f"\nFonte {i}:")
        print("-" * 80)
        print(doc.page_content[:500] + "..." if len(doc.page_content) > 500 else doc.page_content)
        print("-" * 80)
    
    print("\n\n")
    return {"answer": answer, "sources": source_docs}

In [ ]:
# Teste 1: Conceito geral
result1 = ask_and_validate("What is Claude Skills and what is its main purpose?")

In [ ]:
# Teste 2: Instalação
result2 = ask_and_validate("How do I install and set up Claude Skills?")

In [ ]:
# Teste 3: Funcionalidades
result3 = ask_and_validate("What are the main features and capabilities of Claude Skills?")

In [ ]:
# Teste 4: Contribuição
result4 = ask_and_validate("How can developers contribute to the Claude Skills project?")

In [ ]:
# Teste 5: Exemplos de uso
result5 = ask_and_validate("Can you provide examples of how to use Claude Skills?")

## Testes Adicionais

Vamos testar perguntas mais específicas para validar a melhoria do sistema.

In [ ]:
# Teste 6: Pergunta técnica específica
result6 = ask_and_validate("What programming languages or frameworks are required for Claude Skills?")

In [ ]:
# Teste 7: Pergunta sobre arquitetura
result7 = ask_and_validate("How does Claude Skills integrate with existing applications?")

In [ ]:
# Teste 8: Pergunta sobre licença e comunidade
result8 = ask_and_validate("What is the license for Claude Skills and how can I get support?")

## Persistindo o Vectorstore

In [ ]:
vectordb.persist()
print("Vectorstore persistido com sucesso!")

## Conclusão

Este notebook demonstrou como:

1. ✅ Carregar documentação do GitHub (README.md do Claude Skills)
2. ✅ Dividir o documento em chunks otimizados (800 caracteres com overlap de 200)
3. ✅ Criar embeddings usando modelo atualizado (text-embedding-3-small)
4. ✅ Armazenar os vetores em um vectorstore (ChromaDB)
5. ✅ Realizar buscas por similaridade e MMR para melhor diversidade
6. ✅ Usar RAG completo com GPT-4 e prompt customizado
7. ✅ Validar que as respostas estão de acordo com o contexto do documento original

**Melhorias implementadas:**
- ⚡ Chunks menores (800) com maior overlap (200) para preservar contexto
- ⚡ MMR (Maximal Marginal Relevance) para recuperar documentos mais diversos
- ⚡ Modelo GPT-4o-mini para respostas mais precisas
- ⚡ Prompt customizado para instruir melhor o modelo
- ⚡ Recuperação de 5 documentos ao invés de 3
- ⚡ Modelo de embedding mais recente (text-embedding-3-small)

Os testes mostraram que o sistema RAG otimizado é capaz de recuperar informações mais relevantes e gerar respostas significativamente melhores, especialmente para perguntas complexas sobre contribuição e exemplos de uso.